<a href="https://colab.research.google.com/github/hurricane195/Intro-to-Deep-Learning/blob/Homework_5/HW5_P4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Like homework 4, Repeat problem 3, this time try to translate from French to English. Train the model on the entire dataset and evaluate it on the entire dataset. Report training loss, validation loss, and validation accuracy. Also, try some qualitative validation as well, asking the network to generate French translations for some English sentences. Which one seems to be more effective, French-to-English or English-to-French? Compare your results against RNN-based models.

In [ ]:
#Using a modided example of Dr. Tabkhi's "RNN" available at https://github.com/HamedTabkhi/Intro-to-DL/blob/main/RNN.py
#Using a modided example of Dr. Tabkhi's "RNN-CharDataset" available at https://github.com/HamedTabkhi/Intro-to-DL/blob/main/RNN-CharDataset.py
#Using a modided example of Dr. Tabkhi's "shakespeare-loader.py" available at https://github.com/HamedTabkhi/Intro-to-DL/blob/main/shakespeare-loader.py
#Using a modided example of Dr. Tabkhi's "transformer_encoder_nextcharactor" available at https://github.com/HamedTabkhi/Intro-to-DL/blob/main/transformer_encoder_nextcharactor.py
#Using a modided example of Dr. Tabkhi's "TransnNextCharPrediction Positionalencoding" available at https://github.com/HamedTabkhi/Intro-to-DL/blob/main/TransnNextCharPrediction%20Positionalencoding.py
#Using a modided example of Dr. Tabkhi's "TransRegressionPositionalEncoding" available at https://github.com/HamedTabkhi/Intro-to-DL/blob/main/TransRegressionPositionalEncoding.py
#Random help from Chat GPT on formatting, sytntax, etc.
#Random help from Chat Colab AI on formatting, sytntax, etc.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class Vocabulary:
    def __init__(self):
        self.word2index = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2}
        self.index2word = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>"}
        self.word_count = {}
        self.n_words = 3

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word
            self.word_count[word] = 1
            self.n_words += 1
        else:
            self.word_count[word] += 1

def tokenize_and_pad(sentences, vocab):
    max_length = max(len(sentence.split(' ')) for sentence in sentences) + 2
    tokenized_sentences = []
    for sentence in sentences:
        tokens = [vocab.word2index["<SOS>"]] + [vocab.word2index.get(word, vocab.word2index["<PAD>"]) for word in sentence.split(' ')] + [vocab.word2index["<EOS>"]]
        padded_tokens = tokens + [vocab.word2index["<PAD>"]] * (max_length - len(tokens))
        tokenized_sentences.append(padded_tokens)
    return torch.tensor(tokenized_sentences, dtype=torch.long)

In [ ]:
class FrEngDataset(Dataset):
    def __init__(self, pairs):
        self.fr_vocab = Vocabulary()
        self.eng_vocab = Vocabulary()
        self.pairs = []

        for fr, eng in pairs:
            self.fr_vocab.add_sentence(fr)
            self.eng_vocab.add_sentence(eng)
            self.pairs.append((fr, eng))

        self.fr_sentences = [pair[0] for pair in self.pairs]
        self.eng_sentences = [pair[1] for pair in self.pairs]

        self.fr_tokens = tokenize_and_pad(self.fr_sentences, self.fr_vocab)
        self.eng_tokens = tokenize_and_pad(self.eng_sentences, self.eng_vocab)

        self.fr_embedding = torch.nn.Embedding(self.fr_vocab.n_words, 100)
        self.eng_embedding = torch.nn.Embedding(self.eng_vocab.n_words, 100)

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        fr_tokens = self.fr_tokens[idx]
        eng_tokens = self.eng_tokens[idx]
        fr_emb = self.fr_embedding(fr_tokens)
        eng_emb = self.eng_embedding(eng_tokens)
        return fr_tokens, eng_tokens, fr_emb, eng_emb

In [ ]:
french_to_english = [
    ("J'ai froid", "I am cold"),
    ("Tu es fatigué", "You are tired"),
    ("Il a faim", "He is hungry"),
    ("Elle est heureuse", "She is happy"),
    ("Nous sommes amis", "We are friends"),
    ("Ils sont étudiants", "They are students"),
    ("Le chat dort", "The cat is sleeping"),
    ("Le soleil brille", "The sun is shining"),
    ("Nous aimons la musique", "We love music"),
    ("Elle parle français couramment", "She speaks French fluently"),
    ("Il aime lire des livres", "He enjoys reading books"),
    ("Ils jouent au football chaque week-end", "They play soccer every weekend"),
    ("Le film commence à 19 heures", "The movie starts at 7 PM"),
    ("Elle porte une robe rouge", "She wears a red dress"),
    ("Nous cuisinons le dîner ensemble", "We cook dinner together"),
    ("Il conduit une voiture bleue", "He drives a blue car"),
    ("Ils visitent souvent des musées", "They visit museums often"),
    ("Le restaurant sert une délicieuse cuisine", "The restaurant serves delicious food"),
    ("Elle étudie les mathématiques à l'université", "She studies mathematics at university"),
    ("Nous regardons des films le vendredi", "We watch movies on Fridays"),
    ("Il écoute de la musique en faisant du jogging", "He listens to music while jogging"),
    ("Ils voyagent autour du monde", "They travel around the world"),
    ("Le livre est sur la table", "The book is on the table"),
    ("Elle danse avec grâce", "She dances gracefully"),
    ("Nous célébrons les anniversaires avec un gâteau", "We celebrate birthdays with cake"),
    ("Il travaille dur tous les jours", "He works hard every day"),
    ("Ils parlent différentes langues", "They speak different languages"),
    ("Les fleurs fleurissent au printemps", "The flowers bloom in spring"),
    ("Elle écrit de la poésie pendant son temps libre", "She writes poetry in her free time"),
    ("Nous apprenons quelque chose de nouveau chaque jour", "We learn something new every day"),
    ("Le chien aboie bruyamment", "The dog barks loudly"),
    ("Il chante magnifiquement", "He sings beautifully"),
    ("Ils nagent dans la piscine", "They swim in the pool"),
    ("Les oiseaux gazouillent le matin", "The birds chirp in the morning"),
    ("Elle enseigne l'anglais à l'école", "She teaches English at school"),
    ("Nous prenons le petit déjeuner ensemble", "We eat breakfast together"),
    ("Il peint des paysages", "He paints landscapes"),
    ("Ils rient de la blague", "They laugh at the joke"),
    ("L'horloge tic-tac bruyamment", "The clock ticks loudly"),
    ("Elle court dans le parc", "She runs in the park"),
    ("Nous voyageons en train", "We travel by train"),
    ("Il écrit une lettre", "He writes a letter"),
    ("Ils lisent des livres à la bibliothèque", "They read books at the library"),
    ("Le bébé pleure", "The baby cries"),
    ("Elle étudie dur pour les examens", "She studies hard for exams"),
    ("Nous plantons des fleurs dans le jardin", "We plant flowers in the garden"),
    ("Il répare la voiture", "He fixes the car"),
    ("Ils boivent du café le matin", "They drink coffee in the morning"),
    ("Le soleil se couche le soir", "The sun sets in the evening"),
    ("Elle danse à la fête", "She dances at the party"),
    ("Nous jouons de la musique au concert", "We play music at the concert"),
    ("Il cuisine le dîner pour sa famille", "He cooks dinner for his family"),
    ("Ils étudient la grammaire française", "They study French grammar"),
    ("La pluie tombe doucement", "The rain falls gently"),
    ("Elle chante une chanson", "She sings a song"),
    ("Nous regardons un film ensemble", "We watch a movie together"),
    ("Il dort profondément", "He sleeps deeply"),
    ("Ils voyagent à Paris", "They travel to Paris"),
    ("Les enfants jouent dans le parc", "The children play in the park"),
    ("Elle se promène le long de la plage", "She walks along the beach"),
    ("Nous parlons au téléphone", "We talk on the phone"),
    ("Il attend le bus", "He waits for the bus"),
    ("Ils visitent la tour Eiffel", "They visit the Eiffel Tower"),
    ("Les étoiles scintillent la nuit", "The stars twinkle at night"),
    ("Elle rêve de voler", "She dreams of flying"),
    ("Nous travaillons au bureau", "We work in the office"),
    ("Il étudie l'histoire", "He studies history"),
    ("Ils écoutent la radio", "They listen to the radio"),
    ("Le vent souffle doucement", "The wind blows gently"),
    ("Elle nage dans l'océan", "She swims in the ocean"),
    ("Nous dansons au mariage", "We dance at the wedding"),
    ("Il gravit la montagne", "He climbs the mountain"),
    ("Ils font de la randonnée dans la forêt", "They hike in the forest"),
    ("Le chat miaule bruyamment", "The cat meows loudly"),
    ("Elle peint un tableau", "She paints a picture"),
    ("Nous construisons un château de sable", "We build a sandcastle"),
    ("Il chante dans le chœur", "He sings in the choir")
]

In [ ]:
dataset = FrEngDataset(french_to_english)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, nhead, num_layers):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        encoder_layers = nn.TransformerEncoderLayer(hidden_size, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        transformer_output = self.transformer_encoder(embedded)
        output = self.fc(transformer_output[:, -1, :])  # Get the output of the last Transformer block
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))

In [ ]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        encoder_layers = nn.TransformerEncoderLayer(hidden_size, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        transformer_output = self.transformer_encoder(embedded)
        output = self.fc(transformer_output[:, -1, :])  # Get the output of the last Transformer block
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))

In [ ]:
# Hyperparameters
num_layers = 3
nhead = 2
learning_rate = 0.01

hidden_size = 256
input_size = dataset.fr_vocab.n_words
output_size = dataset.eng_vocab.n_words
encoder = Encoder(input_size, hidden_size, nhead, num_layers).to(device)
decoder = Decoder(hidden_size, output_size).to(device)

encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=12):
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    loss = 0

    # Initialize encoder_hidden for training
    encoder_hidden = encoder.initHidden()

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)

    decoder_input = torch.tensor([[dataset.eng_vocab.word2index['<SOS>']]], device=device)
    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()

        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
        if decoder_input.item() == dataset.eng_vocab.word2index['<EOS>']:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

def calculate_accuracy(predicted_indices, target_indices):
    return sum([1 for pred, target in zip(predicted_indices, target_indices) if pred == target]) / len(target_indices)

criterion = nn.NLLLoss()

n_epochs = 110

for epoch in range(n_epochs):
    total_loss = 0
    total_tokens = 0
    correct_tokens = 0

    encoder.train()
    decoder.train()

    for input_tensor, target_tensor, _, _ in dataloader:
        input_tensor = input_tensor[0].to(device)
        target_tensor = target_tensor[0].to(device)

        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        total_loss += loss

        input_length = input_tensor.size(0)
        target_length = target_tensor.size(0)

        predicted_indices = []

        # Initialize encoder_hidden for training
        encoder_hidden = encoder.initHidden()

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)

        decoder_input = torch.tensor([[dataset.eng_vocab.word2index['<SOS>']]], device=device)
        decoder_hidden = encoder_hidden

        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            predicted_indices.append(topi.item())
            decoder_input = topi.squeeze().detach()

            if decoder_input.item() == dataset.eng_vocab.word2index['<EOS>']:
                break

        correct_tokens += calculate_accuracy(predicted_indices, target_tensor.tolist())
        total_tokens += target_length

    val_total_loss = 0
    val_total_tokens = 0
    val_correct_tokens = 0

    encoder.eval()
    decoder.eval()

    with torch.no_grad():
        for input_tensor, target_tensor, _, _ in dataloader:
            input_tensor = input_tensor[0].to(device)
            target_tensor = target_tensor[0].to(device)

            # Initialize encoder_hidden for evaluation
            encoder_hidden = encoder.initHidden()

            input_length = input_tensor.size(0)
            target_length = target_tensor.size(0)

            loss = 0

            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)

            decoder_input = torch.tensor([[dataset.eng_vocab.word2index['<SOS>']]], device=device)
            decoder_hidden = encoder_hidden

            predicted_indices = []

            for di in range(target_length):
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                topv, topi = decoder_output.topk(1)
                predicted_indices.append(topi.item())
                decoder_input = topi.squeeze().detach()

                loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
                if decoder_input.item() == dataset.eng_vocab.word2index['<EOS>']:
                    break

            val_total_loss += loss.item() / target_length
            val_correct_tokens += calculate_accuracy(predicted_indices, target_tensor.tolist())
            val_total_tokens += target_length

    if epoch % 10 == 0:
        print(f'Epoch {epoch}: Train Loss: {total_loss / len(dataloader):.4f}, Train Acc: {correct_tokens / total_tokens:.4f}, Val Loss: {val_total_loss / len(dataloader):.4f}, Val Acc: {val_correct_tokens / val_total_tokens:.4f}')

    if epoch == n_epochs - 1:
        print("\nPrediction Examples:")
        n_examples = 10
        with torch.no_grad():
            for i, (input_tensor, target_tensor, _, _) in enumerate(dataloader):
                input_tensor = input_tensor[0].to(device)
                target_tensor = target_tensor[0].to(device)

                encoder_hidden = encoder.initHidden()

                input_length = input_tensor.size(0)
                target_length = target_tensor.size(0)

                predicted_indices = []

                for ei in range(input_length):
                    encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)

                decoder_input = torch.tensor([[dataset.eng_vocab.word2index['<SOS>']]], device=device)
                decoder_hidden = encoder_hidden

                for di in range(target_length):
                    decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                    topv, topi = decoder_output.topk(1)
                    predicted_indices.append(topi.item())
                    decoder_input = topi.squeeze().detach()

                    if decoder_input.item() == dataset.eng_vocab.word2index['<EOS>']:
                        break

                if i < n_examples:
                    predicted_string = ' '.join([dataset.eng_vocab.index2word[index] for index in predicted_indices if index not in (dataset.eng_vocab.word2index['<SOS>'], dataset.eng_vocab.word2index['<EOS>'], dataset.eng_vocab.word2index['<PAD>'])])
                    target_string = ' '.join([dataset.eng_vocab.index2word[index.item()] for index in target_tensor if index.item() not in (dataset.eng_vocab.word2index['<SOS>'], dataset.eng_vocab.word2index['<EOS>'], dataset.eng_vocab.word2index['<PAD>'])])
                    input_string = ' '.join([dataset.fr_vocab.index2word[index.item()] for index in input_tensor if index.item() not in (dataset.fr_vocab.word2index['<SOS>'], dataset.fr_vocab.word2index['<EOS>'], dataset.fr_vocab.word2index['<PAD>'])])

                    print(f'Input: {input_string}, Target: {target_string}, Predicted: {predicted_string}')

# Calculate overall evaluation results
overall_val_loss = val_total_loss / len(dataloader)
overall_val_accuracy = val_correct_tokens / val_total_tokens
overall_train_loss = total_loss / len(dataloader)
overall_train_accuracy = correct_tokens / total_tokens

print("\nOverall Evaluation Results:")
print(f'Overall Training Loss: {overall_train_loss:.4f}, Overall Training Accuracy: {overall_train_accuracy:.4f}, Overall Validation Loss: {overall_val_loss:.4f}, Overall Validation Accuracy: {overall_val_accuracy:.4f}')

Epoch 0: Train Loss: nan, Train Acc: 0.0300, Val Loss: nan, Val Acc: 0.0303
Epoch 10: Train Loss: nan, Train Acc: 0.0303, Val Loss: nan, Val Acc: 0.0303
Epoch 20: Train Loss: nan, Train Acc: 0.0303, Val Loss: nan, Val Acc: 0.0303
Epoch 30: Train Loss: nan, Train Acc: 0.0303, Val Loss: nan, Val Acc: 0.0303
Epoch 40: Train Loss: nan, Train Acc: 0.0303, Val Loss: nan, Val Acc: 0.0303
Epoch 50: Train Loss: nan, Train Acc: 0.0303, Val Loss: nan, Val Acc: 0.0303
Epoch 60: Train Loss: nan, Train Acc: 0.0303, Val Loss: nan, Val Acc: 0.0303
Epoch 70: Train Loss: nan, Train Acc: 0.0303, Val Loss: nan, Val Acc: 0.0303
Epoch 80: Train Loss: nan, Train Acc: 0.0303, Val Loss: nan, Val Acc: 0.0303
Epoch 90: Train Loss: nan, Train Acc: 0.0303, Val Loss: nan, Val Acc: 0.0303
Epoch 100: Train Loss: nan, Train Acc: 0.0303, Val Loss: nan, Val Acc: 0.0303

Prediction Examples:
Input: Ils écoutent la radio, Target: They listen to the radio, Predicted: 
Input: Ils voyagent à Paris, Target: They travel to Par

In [ ]:
english_sentences = [
"I am cold",
"You are tired",
"He is hungry",
"She is happy",
"We are friends",
"They are students",
"The cat is sleeping",
"The sun is shining",
"We love music",
"He enjoys reading books"
]

In [ ]:
# Define some French sentences for translation
print(" ")
print("English translations for some French sentences:")


# Tokenize and pad the French sentences
tokenized_french_sentences = tokenize_and_pad(english_sentences, dataset.fr_vocab)

# Convert tokenized sentences into tensors
input_tensors = tokenized_french_sentences.to(device)

# Generate English translations
with torch.no_grad():
    for input_tensor in input_tensors:
        # Initialize encoder hidden states
        encoder_hidden = encoder.initHidden()

        input_length = input_tensor.size(0)

        # Pass input through the encoder
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)

        # Initialize decoder input with SOS token
        decoder_input = torch.tensor([[dataset.eng_vocab.word2index['<SOS>']]], device=device)
        decoder_hidden = encoder_hidden

        # Initialize list to store predicted indices
        predicted_indices = []

        # Generate translation
        for _ in range(12):  # Assuming max length is 12
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            predicted_indices.append(topi.item())

            if predicted_indices[-1] == dataset.eng_vocab.word2index['<EOS>']:
                break

            decoder_input = topi.squeeze().detach()

        # Convert predicted indices to English words
        predicted_words = [dataset.eng_vocab.index2word[index] for index in predicted_indices if
                           index not in (dataset.fr_vocab.word2index['<SOS>'], dataset.fr_vocab.word2index['<EOS>'], dataset.fr_vocab.word2index['<PAD>'])]

        # Print the translations
        print("French Sentence:", ' '.join([dataset.fr_vocab.index2word[index.item()] for index in input_tensor if
                                              index.item() not in (dataset.eng_vocab.word2index['<SOS>'], dataset.eng_vocab.word2index['<EOS>'], dataset.eng_vocab.word2index['<PAD>'])]))
        print("English Translation:", ' '.join(predicted_words))

 
English translations for some French sentences:
French Sentence: 
English Translation: 
French Sentence: 
English Translation: 
French Sentence: 
English Translation: 
French Sentence: 
English Translation: 
French Sentence: 
English Translation: 
French Sentence: 
English Translation: 
French Sentence: 
English Translation: 
French Sentence: 
English Translation: 
French Sentence: 
English Translation: 
French Sentence: 
English Translation: 
